In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import astra
astra.set_gpu_index((0, 1, 2, 3))

from AdversarialRegularizer import AdversarialRegulariser
from networks import AlexNet_3D
from networks_FoV import ResNetL2_local
from ellipsgen.generate_data_function import get_batch
import numpy as np
import ellipsgen.CCB_class as CT
import ellipsgen.phantom_class as ph
import matplotlib.pyplot as plt
import time

BATCH_SIZE = 4
LOOPS = 2
IMG_SIZE = 128
STEPS = 100

In [2]:
# Parameter choices. Heuristic in the BWGAN paper: Choose GAMMA as average dual norm of clean image
# LMB should be bigger than product of norm times dual norm.

# For s=0.0, this implies GAMMA =1.0
# For s=1.0, have GAMMA = 10.0 as realisitc value
S = 0.0
LMB = 10.0
GAMMA = 1.0
CUTOFF = 20.0

saves_path = '/export/scratch1/home/voxels-gpu0/codesprint_learned_prior/AdvRegSaves/test2/ell_noisy_' + str(IMG_SIZE)
regularizer = AdversarialRegulariser(saves_path, IMAGE_SIZE=(None, IMG_SIZE, IMG_SIZE, IMG_SIZE, 1), NETWORK=AlexNet_3D,
                                     s=S, cutoff=CUTOFF, lmb=LMB, gamma=GAMMA)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use keras.layers.conv3d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling3d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
No save found


In [3]:
def load_pair(i):
    prefix = '../noisy_' + str(IMG_SIZE) + '/'
    fdk = np.load(prefix + 'FDK/dataset' + str(i) + '.npy')
    gt = np.load(prefix + 'GT/dataset' + str(i) + '.npy')
    
    return fdk, gt


def generate_pairs(amount=1):
    num_angles = 360
    noise = ['Poisson', 2 ** 14] # Do not go below 2 ** 8, lower number in the 2nd argument means more noise
    num_vox = IMG_SIZE
    
    voxels = [num_vox, num_vox, num_vox]
    src_rad = 10
    det_rad = 0

    fdks = np.empty((amount, *voxels))
    gts = np.empty_like(fdks)

    for i in range(amount):
        data_obj = ph.phantom(voxels, '22 Ellipses', num_angles, noise, src_rad, det_rad)
        gts[i, ...] =  data_obj.f

        case = CT.CCB_CT(data_obj)
        fdks[i, ...] = case.do_FDK()
        
        if i == 0:
            plt.figure()
            plt.imshow(gts[i, int(IMG_SIZE/2), ...])
            plt.show()
            
            plt.figure()
            plt.imshow(fdks[i, int(IMG_SIZE/2), ...])
            plt.show()
            
    
    return (gts, fdks)
    

def evaluate(gt, adv):
    # gt, adv = get_batch(eval_data=True, noise_levels=['01', '016'], methods=['EM', 'SGD'])
    regularizer.test(groundTruth=gt, adversarial=adv)

    
def train(steps):
    for k in range(steps):        
#         if k%50 == 0:
#             gt, adv = generate_pairs(amount=1) # batch, x, y, z, channel
#             evaluate(gt, adv)
#         else:
            gts, advs = load_pair(k) # batch, x, y, z, channel
    
            print(k)
            t = time.time()
            regularizer.train(groundTruth=gts, adversarial=advs, learning_rate=LEARNING_RATE)
            print(time.time() - t)
            
    regularizer.save()

In [4]:
LEARNING_RATE = 0.00005
for k in range(LOOPS):
    train(STEPS)

LEARNING_RATE = 0.00002
for k in range(LOOPS):
    train(STEPS)

0
6.754544496536255
1
6.487596750259399
2
5.99824333190918
3
5.665719032287598
4
5.639042139053345
5
5.8206164836883545
6
5.692180395126343
7
5.776052236557007
8
5.7480669021606445
9
5.798701047897339
10
5.50317645072937
11
5.795251131057739
12
5.648846864700317
13
5.879227876663208
14
5.612936496734619
15
5.551056861877441
16
5.575117826461792
17
5.626908302307129
18
5.664790868759155
19
5.932149410247803
20
5.8516762256622314
21
5.601837873458862
22
5.572450637817383
23
5.6839728355407715
24
5.690674066543579
25
5.958280801773071
26
5.702630996704102
27
5.7077295780181885
28
5.798496723175049
29
5.7650346755981445
30
5.6823108196258545
31
5.681435585021973
32
5.738511800765991
33
5.875841856002808
34
5.563677549362183
35
5.760504245758057
36
5.6935036182403564
37
5.7125020027160645
38
5.691011905670166
39
5.659024477005005
40
5.804464340209961
41
5.5742363929748535
42
5.6897454261779785
43
5.643547058105469
44
5.654759407043457
45
5.61041522026062
46
5.695233583450317
47
5.8051431179